In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# https://drive.google.com/drive/folders/<ID>
dir_id = "1DLqYGQKx4tMHDEy85lVfgfxHLC-RNRqJ"

# フォルダ内にあるファイル名とファイルIDを表示する
file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % dir_id}).GetList()
for f in file_list:
  print("name: " + f["title"] + ", id: " + f["id"])

In [0]:
# 鞘師のデータを取得
id = '1h7UdJxcTZjg-KfIHk1NfjAa3dCKAd4Co'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('this_is_sayashi.zip')

In [0]:
! mkdir this_is_sayashi
!unzip this_is_sayashi.zip -d this_is_sayashi

In [0]:
! rm -r this_is_sayashi/__MACOSX/

In [0]:
# 道重のデータを取得
id = '1SvOeSS3iufFJ0w50ruf0BWFyjRNebMQL'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('this_is_sayumi.zip')

In [0]:
! mkdir this_is_sayumi
! unzip this_is_sayumi.zip -d this_is_sayumi

In [0]:
! rm -r this_is_sayumi/__MACOSX/

In [0]:
# 佳林のデータを取得
id = '1bCAcif-NB_H76mYDW4cGKnGB4QoiM8IH'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('this_is_karin.zip')

In [0]:
! mkdir this_is_karin
! unzip this_is_karin.zip -d this_is_karin

In [0]:
! rm -r this_is_karin/__MACOSX/

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
from PIL import Image

In [0]:
from keras.layers import Input
from keras.layers.core import Reshape, Dense, Flatten, Activation
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

def Generator():
    nch = 200
    model_input = Input(shape=[100])
    x = Dense(nch*50*50, kernel_initializer='glorot_normal')(model_input) # 100 -> 200*50*50
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Reshape( [50, 50, nch] )(x) # 200*50*50 -> 50x50x200 (width)x(height)x(channel)
    x = UpSampling2D(size=(2, 2))(x) # 50x50x200 -> 100x100x200
    x = Conv2D(int(nch/2), (3, 3), padding='same', kernel_initializer='glorot_uniform')(x) # 100x100x200 -> 100x100x100
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(int(nch/4), (3, 3), padding='same', kernel_initializer='glorot_uniform')(x) # 100x100x100 -> 100x100x50
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(3, (1, 1), padding='same', kernel_initializer='glorot_uniform')(x) # 100x100x50 -> 100x100x3
    model_output = Activation('sigmoid')(x)
    model = Model(model_input, model_output)
    # model.summary()
    
    return model

In [0]:
from keras.layers import Input
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.models import Model

def Discriminator(shape, dropout_rate=0.25, opt=Adam(lr=1e-4)):
    model_input = Input(shape=shape) # 100x100x3
    x = Conv2D(256, (5, 5), padding = 'same', kernel_initializer='glorot_uniform', strides=(2, 2))(model_input) # 100x100x3 -> 50x50x256
    x = LeakyReLU(0.2)(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(512, (5, 5), padding = 'same', kernel_initializer='glorot_uniform', strides=(2, 2))(x) # 50x50x256 -> 25x25x512
    x = LeakyReLU(0.2)(x)
    x = Dropout(dropout_rate)(x)
    x = Flatten()(x) # 25x25x512 -> 25*25*512
    x = Dense(256)(x)
    x = LeakyReLU(0.2)(x)
    x = Dropout(dropout_rate)(x)
    model_output = Dense(2,activation='softmax')(x) # 256 -> 2
    model = Model(model_input, model_output)
    model.compile(loss='categorical_crossentropy', optimizer=opt)
    model.summary()
    
    return model

In [0]:
from keras.layers import Input
from keras.models import Model

def combined_network(generator, discriminator, opt=Adam(lr=1e-3)):
    gan_input = Input(shape=[100])
    x = generator(gan_input)
    gan_output = discriminator(x)
    model = Model(gan_input, gan_output)
    model.compile(loss='categorical_crossentropy', optimizer=opt)
    # model.summary()
    
    return model

In [0]:
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val

In [0]:
from tqdm import tqdm

def train(step=3000, BATCH_SIZE=128):
    for e in tqdm(range(step)):
        # 1. バッチの学習で利用する画像の選択 
        # バッチサイズの分だけランダムに画像を選択
        image_batch = X_train[np.random.randint(0,X_train.shape[0],size=BATCH_SIZE),:,:,:]
        
        # バッチサイズの分だけランダムにノイズを生成し、generatorにより画像を生成
        noise_gen = np.random.uniform(0,1,size=[BATCH_SIZE,100])
        generated_images = generator.predict(noise_gen)
        
        # 2. Discriminatorの学習をonに切り替える
        # Discriminatorが学習するように変更
        make_trainable(discriminator,True)
        
        # 3. Generatorによる生成画像を用いてDiscriminatorの学習
        # X = (バッチサイズ分のデータセットの画像, バッチサイズ分の生成画像)
        X = np.concatenate((image_batch, generated_images))
        
        # y = (バッチサイズ分のTrue(本物), バッチサイズ分のFalse(偽物))
        y = np.zeros([2*BATCH_SIZE,2])
        y[:BATCH_SIZE,1] = 1
        y[BATCH_SIZE:,0] = 1      
        
        # Discriminatorのtrain
        discriminator.train_on_batch(X,y)
        
        # 4. Discriminatorの学習をoffに切り替える
        # Discriminatorが学習しないように変更
        make_trainable(discriminator,False)
    
        # 5. Generatorの学習
        # バッチサイズの分だけランダムにノイズを生成
        noise_gen = np.random.uniform(0,1,size=[BATCH_SIZE,100])
        
        # y = (バッチサイズ分のTrue(本物))
        # 実際には生成した画像なのでDiscriminatorとしては偽物と判断すべきだが、Genaratorの学習なので生成した画像を本物と判断するように学習させる
        y2 = np.zeros([BATCH_SIZE,2])
        y2[:,1] = 1
        
        # Generatorのtrain
        GAN.train_on_batch(noise_gen, y2 )

In [0]:
# データのロード

# サイズを合わせる
from PIL import Image, ImageOps

list_pic_sayashi = []
for p in list_pictures('this_is_sayashi'):
    list_pic_sayashi.append(load_img(p).resize((100, 100), Image.LANCZOS))

list_pic_sayashi_2 = []
for p in range(len(list_pic_sayashi)):
    list_pic_sayashi_2.append(ImageOps.mirror(list_pic_sayashi[p]))
    
list_pic_sayashi.extend(list_pic_sayashi_2)

X0 = np.empty((len(list_pic_sayashi),100,100,3))
Y0 = []

#　鞘師の画像(0)
for i, picture in enumerate(list_pic_sayashi):
    img_is_sayashi = img_to_array(picture)
    X0[i,:,:,:] = img_is_sayashi
    
    Y0.append(0)
Y0 = np.array(Y0)
list_pic_sayashi[260]

In [0]:
# サイズを合わせる
from PIL import Image

list_pic_sayumi = []
for p in list_pictures('this_is_sayumi/this_is_sayumi'):
    list_pic_sayumi.append(load_img(p).resize((100, 100), Image.LANCZOS))

list_pic_sayumi_2 = []
for p in range(len(list_pic_sayumi)):
    list_pic_sayumi_2.append(ImageOps.mirror(list_pic_sayumi[p]))

list_pic_sayumi.extend(list_pic_sayumi_2)

X1 = np.empty((len(list_pic_sayumi),100,100,3))
Y1 = []

#　道重の画像(1)
for i, picture in enumerate(list_pic_sayumi):
    img_is_sayumi = img_to_array(picture)
    X1[i,:,:,:] = img_is_sayumi
    
    Y1.append(1)
Y1 = np.array(Y1)
list_pic_sayumi[0]

In [0]:
# サイズを合わせる
from PIL import Image

list_pic_karin = []
for p in list_pictures('this_is_karin/tihs_is_karin'):
    list_pic_karin.append(load_img(p).resize((100, 100), Image.LANCZOS))

list_pic_karin_2 = []
for p in range(len(list_pic_karin)):
    list_pic_karin_2.append(ImageOps.mirror(list_pic_karin[p]))

list_pic_karin.extend(list_pic_karin_2)

X2 = np.empty((len(list_pic_karin),100,100,3))
Y2 = []

#　佳林の画像(2)
for i, picture in enumerate(list_pic_karin):
    img_is_karin = img_to_array(picture)
    X2[i,:,:,:] = img_is_karin
    
    Y2.append(2)
Y2 = np.array(Y2)
list_pic_karin[0]

In [0]:
X_train = np.concatenate((X0, X1, X2))
array_to_img(X_train[1900])

In [0]:
X_train = X_train/255
X_train

In [0]:
# それぞれのネットワークのインスタンスを生成
generator = Generator()
discriminator = Discriminator(X_train.shape[1:])
make_trainable(discriminator, False)
GAN = combined_network(generator, discriminator)

In [0]:
# train関数で学習を行うstepを大きくすると学習をより多く行います
train(step=20000, BATCH_SIZE=32)

In [0]:
from google.colab import files

x = 3

noise = np.random.uniform(0,1,size=[x, 100])
generated_images = generator.predict(noise)

img = []

for i in range(x):
    img.append(array_to_img(generated_images[i],scale=True))
    img[i].save('img{0}.jpg'.format(str(i)))
    files.download('img{0}.jpg'.format(str(i)))

In [0]:
generator.save('sayashi3g.h5')
files.download('sayashi3g.h5')
discriminator.save('sayashi3d.h5')
files.download('sayashi3d.h5')

In [0]:
train(step=10000, BATCH_SIZE=32)